# En este notebook se leen fotogramas de un video y se obtiene la foto (tambien la guarda) y predicción del label box

In [15]:
import cv2
import time
import warnings; warnings.simplefilter('ignore')
import os

In [16]:
# Lanzamos nuestro otro notebook donde cargamos nuestro modelo y definimos las multiples funciones que usaremos
start_time = time.time()
%run FuncionesYolo.ipynb
print("--- %s seconds ---" % (time.time() - start_time))

--- 10.63559079170227 seconds ---


In [23]:
# definimos los inputs fijos de nuestro modelo (tamaño de la fotografia, anchors y el threshhold):
input_w, input_h = 128, 128
anchors = [[116,90,  156,198,  373,326],  [30,61, 62,45,  59,119], [10,13,  16,30,  33,23]]
class_threshold = 0.6
c= 30 # los fotograms que queremos queignore antes seleccionar el siguiente. 30 equivale a avanzar 1 segundo. 15 equivale a 0,5 segundos
eliminamos = True #Si marcamos True, las imagenes seran eliminadas al instante. Las tenemos que crear temporalmente por el modelo.

In [33]:

start_time = time.time()

#Importamos nuestro video
cap = cv2.VideoCapture('IMG_3644.MOV')
count = 0

while cap.isOpened():
    ret, frame = cap.read() #Accedemos a cada frame

    if ret:
        start_time1 = time.time()
        name='PruebaNotebook/frame'+str(count)+'.jpg' #Definimos donde guardamos nuestras fotografias
        cv2.imwrite(name, frame)
        #print(name + ' creado')
        image, image_w, image_h = load_image_pixels(name, (net_w, net_w)) #Obtenemos nuestas imagenes con el formato deseado
        yolos = yolov3.predict(image) #Hacemos la predicción de nuestro modelo
        boxes = list()
        for i in range(len(yolos)):
            boxes += decode_netout(yolos[i][0], anchors[i], obj_thresh,  net_h, net_w) #decodificamos el outoput de la red
        correct_yolo_boxes(boxes, image_h, image_w, net_h, net_w) #Corregimos el tamaño de los bounding boxes
        do_nms(boxes, nms_thresh) #Eliminamos las non-maximal boxes
        # get the details of the detected objects
        v_boxes, v_labels, v_scores = get_boxes(boxes, labels, class_threshold) #Obtenemos los detalles del objeto detectado
        for i in range(len(v_boxes)):
            print(v_labels[i], v_scores[i])
            print(v_boxes[i].ymin, v_boxes[i].ymax, v_boxes[i].xmin, v_boxes[i].xmax)
        #draw_boxes(name, v_boxes, v_labels, v_scores) #Dibujamos nuestro fotografia con el objeto labelizado       
    
        count += c # Seleccionamos cuanto queremos que avance para seleccionar otro frame. Con 30 fps, avancamos 1 segundo
        cap.set(1, count)
        
        if eliminamos == True: #Marcamos si queremos eliminar o no las imagenes creadas.
            os.remove(name)
        
        if (count/30)%10 == 0:
            print("\n \n Estamos en el fotograma "+str(count))
            print("Hemos estudiado " + str(count/30) + " segundos de video \n \n")
        
        print("--- %s seconds ---" % (time.time() - start_time1))
    else:
        cap.release()
        break
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.31958794593811035 seconds ---
--- 0.3181493282318115 seconds ---
ArrozExtra 97.26245403289795
801 1635 399 1745
--- 0.344590425491333 seconds ---
ArrozExtra 98.82999062538147
197 1591 387 1787
--- 0.34658312797546387 seconds ---
ArrozExtra 98.13753962516785
126 1529 322 1778
--- 0.3366246223449707 seconds ---
ArrozExtra 95.55284976959229
-168 1292 191 1654
--- 0.3291201591491699 seconds ---
ArrozExtra 94.3993628025055
-18 1400 489 1934
--- 0.339092493057251 seconds ---
ArrozExtra 93.96699070930481
-2 1654 352 1972
--- 0.33794569969177246 seconds ---
ArrozExtra 99.06650185585022
140 1711 299 1875
--- 0.3914811611175537 seconds ---
ArrozExtra 98.9432156085968
118 1713 256 1887

 
 Estamos en el fotograma 300
Hemos estudiado 10.0 segundos de video 
 

--- 0.34354496002197266 seconds ---
ArrozExtra 93.06071400642395
-103 1457 160 1821
--- 0.34108591079711914 seconds ---
--- 0.3041853904724121 seconds ---
--- 0.3041999340057373 seconds ---
--- 0.3176600933074951 seconds ---
--- 0.3331

In [11]:
count/30

88.0

In [14]:
count/60

44.0